# Badem: Context-Aware Spell Checking Seq2Seq Model for Turkish

The notebook consists of two parts:

- First part is where we use [TS Wikipedia Corpus](https://tscorpus.com/corpora/ts-wikipedia-corpus/), that consists of 45,245,304 million tokens, to generate the dataset to train the model.
- Second part is the actual training.

## Synthetic Error Generation

- This part is aimed to generate synthetic errors with 1-2-3 Edit Distances from the original words with error characters being selected based on the neighborhood of original characters in Turkish Q keyboard. 

- Generation consists of 3 different operations: replacement, insertion, deletion.

### If generated dataset is provided, you can skip this part.

In [1]:
using StatsBase;

In [4]:
line_2_sentence_tokenizer = (x)  -> x # split(x, ('.'))

tokenizer = split

lower_chars = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', 'z', 'ğ', 'ı', 'ü', 'ö', 'ç', 'ş', 'w', 'x', 'q', '.', ',', ' ']
upper_chars = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'İ', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z', 'Ğ', 'I', 'Ü', 'Ö', 'Ç', 'Ş', 'W', 'X', 'Q', '.', ',', ' '];


function lowercase_tr(str)
    new_word = []
    
    for c in str
        locs = findall(x-> c==x, upper_chars)
        if length(locs) != 0
            push!(new_word, lower_chars[locs[1]])
        else
            push!(new_word, c)
        end
    end
    
    join(new_word)
end

lowercase_tr (generic function with 1 method)

In [5]:
filename = "ts_wikipedia-export.txt"

io = open(filename);
lines = readlines(io)
close(io)

sentences = lowercase_tr.(lines)
seperated_sentences = tokenizer.(sentences);

In [12]:
#=

# Neighbor lookup calculation
# Ran only once, it is not necessary to be calculated again. This code snippet below is to calculate 
# the distance between each letter.

key_positions = Dict(
        'q' => (1,1),
        'w' => (2,1),
        'e' => (3,1),
        'r' => (4,1),
        't' => (5,1),
        'y' => (6,1),
        'u' => (7,1),
        'ı' => (8,1),
        'o' => (9,1),
        'p' => (10,1),
        'ğ' => (11,1),
        'ü' => (12,1),
        'a' => (1.25,2),
        's' => (2.25,2),
        'd' => (3.25,2),
        'f' => (4.25,2),
        'g' => (5.25,2),
        'h' => (6.25,2),
        'j' => (7.25,2),
        'k' => (8.25,2),
        'l' => (9.25,2),
        'ş' => (10.25,2),
        'i' => (11.25,2),
        'z' => (1.75,3),
        'x' => (2.75,3),
        'c' => (3.75,3),
        'v' => (4.75,3),
        'b' => (5.75,3),
        'n' => (6.75,3),
        'm' => (7.75,3),
        'ö' => (8.75,3),
        'ç' => (9.75,3)  
); 

neighbor_lookup = Dict()

for (c, pos) in key_positions
    neighbors = collect(key_positions)

    sort!(neighbors, lt= (x,y) -> sum((x[2].-pos).^2) < sum((y[2].-pos).^2))
    neighbor_lookup[c] = map(x->x[1], neighbors[2:6])
end
=#

# 5 closest neighbors of each key
neighbor_lookup = Dict('n' => ['m', 'b', 'h', 'j', 'k'], 'f' => ['d', 'g', 'r', 'c', 'v'], 'w' => ['e', 'q', 's', 'a', 'd'], 'ç' => ['ö', 'ş', 'l', 'k', 'i'], 'd' => ['f', 's', 'e', 'c', 'x'], 'e' => ['w', 'r', 'd', 's', 'f'], 'o' => ['ı', 'p', 'l', 'k', 'ş'], 'ı' => ['o', 'u', 'k', 'j', 'l'], 'h' => ['j', 'g', 'y', 'n', 'b'], 'y' => ['t', 'u', 'h', 'g', 'j'], 's' => ['d', 'a', 'w', 'z', 'x'], 'r' => ['e', 't', 'f', 'd', 'g'], 't' => ['y', 'r', 'g', 'f', 'h'], 'j' => ['h', 'k', 'u', 'n', 'm'], 'k' => ['j', 'l', 'ı', 'ö', 'm'], 'q' => ['w', 'a', 's', 'e', 'z'], 'ğ' => ['p', 'ü', 'i', 'ş', 'o'], 'ş' => ['i', 'l', 'p', 'ç', 'ğ'], 'i' => ['ş', 'ğ', 'ü', 'p', 'ç'], 'ö' => ['ç', 'm', 'k', 'l', 'j'], 'a' => ['s', 'q', 'z', 'w', 'x'], 'c' => ['x', 'v', 'f', 'd', 's'], 'p' => ['o', 'ğ', 'ş', 'l', 'i'], 'ü' => ['ğ', 'i', 'p', 'ş', 'l'], 'm' => ['n', 'ö', 'j', 'k', 'h'], 'z' => ['x', 's', 'a', 'd', 'c'], 'g' => ['f', 'h', 't', 'v', 'b'], 'x' => ['c', 'z', 'd', 's', 'f'], 'u' => ['ı', 'y', 'j', 'h', 'k'], 'l' => ['k', 'ş', 'o', 'ç', 'ö'], 'v' => ['c', 'b', 'f', 'g', 'd'], 'b' => ['n', 'v', 'h', 'g', 'f']);

In [13]:
function apply_replacement(word, dist=2)
    if dist > length(word)
        return word
    end
    
    indices = sample(1:length(word), dist, replace = false)   
    chars =   rand(1:5, dist) 

    new_word = []
    j = 1
        
    for (i, c) in enumerate(word)
        if i in indices
            new_char_lookup = get(neighbor_lookup, c, c)
            if length(new_char_lookup) == 5
                push!(new_word, new_char_lookup[chars[j]])
            else
                push!(new_word, c)
            end
            
            j = j + 1
        else
            push!(new_word, c)
        end
    end
    
    return join(new_word)
end

function apply_insert(word, dist=2)
    
    indices = sample(1:(length(word)+dist), dist, replace = false)   
    chars =   rand(1:5, dist) 

    new_word = []
    j = 1
    
    for (i, c) in enumerate(word)
        if i in indices
            new_char_lookup = get(neighbor_lookup, c, c)
            if length(new_char_lookup) == 5
                push!(new_word, new_char_lookup[chars[j]])
            end
            push!(new_word, c)
            j = j + 1
            
        else
            push!(new_word, c)
        end
    end
    
    return join(new_word)
end

function apply_deletion(word, dist=2)
    if dist > length(word)
        return word
    end
    
    indices = sample(1:length(word), dist, replace = false)   
    
    new_word = []
    
    for (i, c) in enumerate(word)
        if !(i in indices)
            push!(new_word, c)            
        end
    end
    
    return join(new_word)
end


#Currently not in use
function apply_englishification(word)  
    return replace(word, "ı"=>"i", "ç"=>"c", "ş"=>"s", "ğ"=>"g", "ü"=>"u", "ö"=>"o")
end

apply_englishification (generic function with 1 method)

In [19]:
edit_methods = [apply_deletion, apply_insert, apply_replacement]

errored_sentences = []
correct_words = []

window_size = 2

for sentence in seperated_sentences[:]
    if length(sentence) < window_size * 2 + 1
        continue
    end
    
    
    for j in 1: Int(floor((length(sentence))/(window_size*2+1)))
        i = (j-1) * (2*window_size + 1) + window_size + 1
        
        if length(sentence[i]) < 3
            continue
        end

        push!(correct_words, sentence[i])
        sentence[i] = "∑" * edit_methods[rand(1:3)](sentence[i], rand(1:3)) * "∑"

        push!(errored_sentences, "Ω"*join(sentence[i-window_size:i+window_size], " ")*"Ω")
    end
end

In [24]:
split_num = Int(trunc(length(errored_sentences) * 4/5))

outfile = "shorter-train-sentences-" * filename
f = open(outfile, "w")

for i in errored_sentences[1:split_num] # or for note in notes
    println(f, i)
end

close(f);

outfile = "shorter-train-words-" * filename
f = open(outfile, "w")

for i in correct_words[1:split_num] # or for note in notes
    println(f, i)
end

close(f);


outfile = "shorter-test-sentences-" * filename
f = open(outfile, "w")

for i in errored_sentences[split_num+1:end] # or for note in notes
    println(f, i)
end

close(f);

outfile = "shorter-test-words-" * filename
f = open(outfile, "w")

for i in correct_words[split_num+1:end] # or for note in notes
    println(f, i)
end

close(f);

## S2S Model For Spell Checking

The model uses characters as tokens.

In [25]:
lower_chars = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', 'z', 'ğ', 'ı', 'ü', 'ö', 'ç', 'ş', 'w', 'x', 'q', '.', ',', ' ']
upper_chars = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'İ', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z', 'Ğ', 'I', 'Ü', 'Ö', 'Ç', 'Ş', 'W', 'X', 'Q', '.', ',', ' '];

In [26]:
using Knet, Test, Base.Iterators, IterTools, Random # , LinearAlgebra, StatsBase
using AutoGrad: @gcheck  # to check gradients, use with Float64
Knet.atype() = KnetArray{Float32}  # determines what Knet.param() uses.
#macro size(z, s); esc(:(@assert (size($z) == $s) string(summary($z),!=,$s))); end # for debugging

In [27]:
char_to_token_order = ['Ω', '∑', 'π', ';', '!', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', 'z', 'ğ', 'ı', 'ü', 'ö', 'ç', 'ş', 'w', 'x', 'q', '.', ',', ' ']

EMBED_SIZE = length(char_to_token_order)

function char_to_token(str)
    tokens = []
    for char in str
       res = findfirst(x->x==char, char_to_token_order)

       if res != nothing
            push!(tokens, res)
       else
            push!(tokens, 3)
       end
    end
    
    return tokens
end

char_to_token (generic function with 1 method)

In [28]:
struct TextReader
    filename::String
end

window = 2
EOS = 1

function Base.iterate(r::TextReader, s=nothing)
    if s == nothing
        f = open(r.filename, "r")
        s = (f, readline(f));
    elseif eof(s[1])
        close(s[1]);
        return nothing;
    end
    (f, line) = s;
    
    seperated_line = split(line)
    
    
    
    tokens_in_line = char_to_token.(line)
    
    return tokens_in_line, (f, readline(f))     
end

Base.IteratorSize(::Type{TextReader}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{TextReader}) = Base.HasEltype()
Base.eltype(::Type{TextReader}) = Vector{Int}

In [29]:
struct Embed; w; end

function Embed(vocabsize::Int, embedsize::Int)
    # Your code here
    Embed(param(embedsize, vocabsize))
end

function (l::Embed)(x)
    # Your code here
    l.w[:, x]
end

struct Linear; w; b; end

function Linear(inputsize::Int, outputsize::Int)
    # Your code here
    Linear(param(outputsize, inputsize), param0(outputsize, 1))
end

function (l::Linear)(x)
    # Your code here
    l.w * x .+ l.b
end

function mask!(a,pad)
    # Your code here
    for i in 1:size(a)[1]
       for j in Iterators.reverse(2:size(a)[2])
            if a[i,j-1] == pad
                a[i,j] = 0
            else
                break
            end
        end
    end
 
    return a
end

mask! (generic function with 1 method)

In [30]:
EOS = 1

struct MTData
    src::TextReader        # reader for source language data
    tgt::TextReader        # reader for target language data
    batchsize::Int         # desired batch size
    maxlength::Int         # skip if source sentence above maxlength
    batchmajor::Bool       # batch dims (B,T) if batchmajor=false (default) or (T,B) if true.
    bucketwidth::Int       # batch sentences with length within bucketwidth of each other
    buckets::Vector        # sentences collected in separate arrays called buckets for each length range
    batchmaker::Function   # function that turns a bucket into a batch.
end

function MTData(src::TextReader,  tgt::TextReader; batchmaker = arraybatch, batchsize = 128, maxlength = typemax(Int),
                batchmajor = false, bucketwidth = 10, numbuckets = min(128, maxlength ÷ bucketwidth))
    buckets = [ [] for i in 1:numbuckets ] # buckets[i] is an array of sentence pairs with similar source sentence length
    MTData(src, tgt, batchsize, maxlength, batchmajor, bucketwidth, buckets, batchmaker)
end

Base.IteratorSize(::Type{MTData}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{MTData}) = Base.HasEltype()
Base.eltype(::Type{MTData}) = NTuple{2}

function Base.iterate(d::MTData, state=nothing)
    if state == nothing
        for b in d.buckets; empty!(b); end
    end
    bucket,ibucket = nothing,nothing
    while true
        iter = (state === nothing ? iterate(Iterators.zip(d.src, d.tgt)) : iterate(Iterators.zip(d.src, d.tgt), state))
        if iter === nothing
            ibucket = findfirst(x -> !isempty(x), d.buckets)
            bucket = (ibucket === nothing ? nothing : d.buckets[ibucket])
            break
        else
            sent, state = iter
            if length(sent[1]) > d.maxlength || length(sent[1]) == 0; continue; end
            
            ibucket = min(1 + (length(sent[1])-1) ÷ d.bucketwidth, length(d.buckets))
            bucket = d.buckets[ibucket]
            push!(bucket, sent)
            
            if length(bucket) === d.batchsize; break; end
        end
    end
    if bucket === nothing; return nothing; end
    batchsize = length(bucket)
    batch = arraybatch(d, bucket)
    empty!(bucket)
    
    return batch, state #a, state
end

function arraybatch(d::MTData, bucket)
    # Your code here
    src_max = maximum(x -> length(x[1]), bucket);
    tgt_max = maximum(x -> length(x[2]), bucket);
    
    batch_x = fill(EOS, d.batchsize, src_max)
    batch_y = fill(EOS, d.batchsize, tgt_max+2)
    
    for (i, (src_senten, tgt_senten)) in enumerate(bucket)
        for j in 1:length(src_senten)
            batch_x[i, j+(src_max-length(src_senten))] = src_senten[j]
        end
    end
    
    for (i, (src_senten, tgt_senten)) in enumerate(bucket)
        for j in 1:length(tgt_senten)
            batch_y[i, j+1] = tgt_senten[j]
        end
    end
    
    return (batch_x, batch_y)
end


arraybatch (generic function with 1 method)

In [31]:
EOS = 1

struct S2S_v1
    srcembed::Embed     # source language embedding
    encoder::RNN        # encoder RNN (can be bidirectional)
    tgtembed::Embed     # target language embedding
    decoder::RNN        # decoder RNN
    projection::Linear  # converts decoder output to vocab scores
    dropout::Real       # dropout probability to prevent overfitting
end

function S2S_v1(hidden::Int,         # hidden size for both the encoder and decoder RNN
                srcembsz::Int,       # embedding size for source language
                tgtembsz::Int;      # embedding size for target language
                layers=1,            # number of layers
                bidirectional=false, # whether encoder RNN is bidirectional
                dropout=0)           # dropout probability
    # Your code here
    S2S_v1(Embed(EMBED_SIZE, srcembsz), RNN(srcembsz, hidden; bidirectional, dropout, numLayers=layers, atype= Knet.atype()), Embed(EMBED_SIZE, tgtembsz), RNN(tgtembsz, hidden; dropout, numLayers=(layers*(bidirectional ? 2 : 1)), atype=Knet.atype()), Linear(hidden, EMBED_SIZE), dropout)
end

S2S_v1

In [32]:
@doc RNN

function (s::S2S_v1)(src, tgt; average=true)
    # Your code here
    s.encoder.h = 0
    s.encoder(s.srcembed(src))

    s.decoder.h = s.encoder.h

    z = s.decoder(s.tgtembed(tgt[:, 1:end-1]))
    
    pred = s.projection(reshape(z, :, size(z)[2]*size(z)[3]))
    ans  = reshape(mask!(tgt[:,2:end], EOS), size(tgt[:, 2:end])[1]*size(tgt[:,2:end])[2])
        
    return nll(pred, ans; average)
end

In [33]:
@info "Testing S2S_v1"
Random.seed!(1)
model = S2S_v1(512, 512, 512, layers=2, bidirectional=true, dropout=0.2)

train_sentences =  TextReader("shorter-train-sentences-ts_wikipedia-export.txt")
test_sentences  =  TextReader("shorter-test-sentences-ts_wikipedia-export.txt")

train_words = TextReader("shorter-train-words-ts_wikipedia-export.txt")
test_words  = TextReader("shorter-test-words-ts_wikipedia-export.txt")

dtrn = MTData(train_sentences, train_words)
ddev = MTData(test_sentences,  test_words)

(x, y) = first(dtrn)

model(x,y; average=false)

┌ Info: Testing S2S_v1
└ @ Main In[33]:1


LoadError: InterruptException:

In [34]:
function loss(model, data; average=true)
    # Your code here
    x, y = first(data)
    🥲 = model(x, y; average = average) 
    
    for (x, y) in drop(data, 1)
        🥲 = model(x, y; average = average) .+ 🥲
    end
    
    return 🥲 ./ (average ? length(data) : 1)
end

loss (generic function with 1 method)

In [37]:
#@info "Testing loss"
#@test loss(model, dtst, average=false) == (1.0427646f6, 105937)
# Your loss can be slightly different due to different ordering of words in the vocabulary.
# The reference vocabulary starts with eos, unk, followed by words in decreasing frequency.
# Also, because we do not mask src, different batch sizes may lead to slightly different
# losses. The test above gives (1.0430301f6, 105937) with batchsize==1.

function train!(model, trn, dev, tst...)
    bestmodel, bestloss = deepcopy(model), loss(model, dev)
    progress!(adam(model, trn), steps=100) do y
        losses = [ loss(model, d) for d in (dev,tst...) ]
        if losses[1] < bestloss
            bestmodel, bestloss = deepcopy(model), losses[1]
        end
        return (losses...,)
    end
    return bestmodel
end

epochs = 10
ctrn   = collect(dtrn)
trnx10 = collect(flatten(shuffle!(first(ctrn, 2500)) for i in 1:epochs))
trn20  = ctrn[1:20]
dev38  = collect(ddev)
# Uncomment this to train the model (This takes about 30 mins on a V100, 60 mins on a T4):

7120-element Vector{Tuple{T, T} where T}:
 ([1 1 … 23 1; 1 1 … 38 1; … ; 1 1 … 3 1; 1 1 … 6 1], [1 10 … 1 1; 1 23 … 1 1; … ; 1 12 … 1 1; 1 18 … 1 1])
 ([1 1 … 19 1; 1 1 … 6 1; … ; 1 1 … 10 1; 1 1 … 19 1], [1 18 … 1 1; 1 23 … 1 1; … ; 1 21 … 1 1; 1 7 … 1 1])
 ([1 1 … 18 1; 1 1 … 6 1; … ; 1 1 … 38 1; 1 1 … 10 1], [1 10 … 1 1; 1 6 … 1 1; … ; 1 7 … 1 1; 1 23 … 1 1])
 ([1 1 … 22 1; 1 1 … 22 1; … ; 1 1 … 38 1; 1 1 … 31 1], [1 23 … 1 1; 1 27 … 1 1; … ; 1 14 … 1 1; 1 20 … 1 1])
 ([1 1 … 19 1; 1 1 … 6 1; … ; 1 24 … 22 1; 1 1 … 14 1], [1 23 … 1 1; 1 7 … 1 1; … ; 1 16 … 1 1; 1 13 … 1 1])
 ([1 1 … 30 1; 1 1 … 19 1; … ; 1 1 … 19 1; 1 1 … 23 1], [1 12 … 1 1; 1 27 … 1 1; … ; 1 25 … 1 1; 1 6 … 1 1])
 ([1 1 … 30 1; 1 1 … 18 1; … ; 1 1 … 6 1; 1 1 … 22 1], [1 14 … 1 1; 1 20 … 1 1; … ; 1 14 … 1 1; 1 18 … 1 1])
 ([1 1 … 31 1; 1 39 … 19 1; … ; 1 1 … 14 1; 1 27 … 14 1], [1 16 … 1 1; 1 19 … 1 1; … ; 1 16 … 1 1; 1 16 … 1 1])
 ([1 1 … 39 1; 1 1 … 19 1; … ; 1 1 … 19 1; 1 33 … 6 1], [1 3 … 1 1; 1 3 … 1 1; … ; 1 1

In [38]:
model  = train!(model, trnx10, dev38, trn20)
# Uncomment this to save the model:
Knet.save("shorter-s2s_v1.jld2","model", model)
# Uncomment this to load the model:

┣████████████████████┫ [100.00%, 25000/25000, 04:40:10/04:40:10, 1.49i/s] (1.4826353f0, 1.2705247f0)


In [49]:
function (s::S2S_v1)(src::Matrix{Int}; stopfactor = 3)
    # Your code here
    s.encoder.h = 0
    s.encoder(s.srcembed(src))
    
    s.decoder.h = value(s.encoder.h)
    in_decoder = s.tgtembed(fill(EOS, size(src)[1] ))
    
    sentences = []
    num_steps = 0
    
    ended_ones = fill(false, size(src)[1])
    
    while true
        z = s.decoder(in_decoder)
        scores = s.projection(z)
        indices = getindex.(argmax(scores, dims=1), 1)
        pred = reshape(indices, 1, length(indices))
        
        ended_ones = reshape(collect(pred .== EOS), size(src)[1] ) .| ended_ones
        
        pred[ended_ones] .= EOS
        
        push!(sentences, pred)
        
        num_steps += 1
        in_decoder = z
        
        if num_steps >= stopfactor * size(src,2) || findfirst(x->!x, ended_ones) == nothing
            break;
        end
    end
 
    return (char_to_token_order[transpose(reduce(vcat, sentences))])
end

In [52]:
# Utility to convert int arrays to sentence strings
function int2str(y,vocab)
    y = vec(y)
    ysos = findnext(w->!isequal(w,vocab.eos), y, 1)
    ysos == nothing && return ""
    yeos = something(findnext(isequal(vocab.eos), y, ysos), 1+length(y))
    join(vocab.i2w[y[ysos:yeos-1]], " ")
end

# Uncomment and run these lines if you get a "CUDNNError: CUDNN_STATUS_INTERNAL_ERROR (code 4)" error from the cell below.
# Knet.save("s2s_v1.jld2","model",model)
model = Knet.load("shorter-s2s_v1.jld2","model")

@info "Generating some translations"
#d = MTData(tr_dev, en_dev, batchsize=1) |> collect

(src,tgt) = first(ddev)

out = model(src)

print(out)

['b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω'; 'b' 's' 'o' 'e' 'a' 'Ω';

┌ Info: Generating some translations
└ @ Main In[52]:14
